## Introduction

This notebook deals with classification of separable and entangled states.

In [10]:
!pip install numpy
!pip install qutip
!pip install cirq
!pip install pyod
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached importlib_metadata-3.10.1-py3-none-any.whl (14 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.12.0
    Uninstalling importlib-metadata-4.12.0:
      Successfully uninstalled importlib-metadata-4.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: http

## PPT Criterion

Here, we code the required PPT criterion.

In [6]:
def PPT(matrix: np.ndarray, n: int, m: int) -> int:
    rho = qutip.Qobj(matrix)
    rho.dims = [[n, n], [m, m]]
    rho_out = qutip.partial_transpose(rho, [0, 1])
    if min(rho_out.eigenenergies()) >= 0:
        return 0 # Separable
    else:
        return 1 # Entangled

def check_PPT(matrices: np.ndarray, n: int, m: int) -> list:
    res = []
    for matrix in matrices:
        res.append(PPT(matrix, n, m))
    return res

In [5]:
def embedding_matrices(matrices: np.ndarray) -> list:
    embedding = []
    for matrix in matrices:
        embedding.append(embed(matrix))
    return np.array(embedding)

## Data Embedding and Generation

The following deals with generation and embedding of data.

In [7]:
import qutip
import numpy as np
import random
from matplotlib import pyplot as plt
from cirq.linalg import is_unitary
from cirq.qis.states import validate_density_matrix


def embed(matrix: np.ndarray, e: bool = True) -> list:
    if e == False:
        return matrix
        
    x, y = matrix.shape

    # Validate the given density matrix
    try:
        validate_density_matrix(matrix, qid_shape=x)
    except:
        print(f"Not valid density matrix: {matrix}")
        exit(0)
    matrix = matrix.tolist()

    # Flatten and embed the density matrix by taking care that,
    # it has (n - 1) diagonal values and upper traingular matrix.
    flattened = []
    for i in range(x):
        for j in range(y):
            if i < j:
                flattened.append(matrix[i][j])

    # Embed the real and complex values respectively.
    embedding = []
    for i in flattened:
        embedding.extend([i.real, i.imag])

    # Embed diagonal matrices
    for i in range(1, x):
        embedding.append(matrix[i][i].real)

    return embedding


def get_separable_state(n: int, m: int, max_len: int = 50) -> np.ndarray:
    l = random.randint(1, max_len)
    prob = np.array([random.random() for _ in range(l)])
    prob = prob / np.sum(prob)

    separable_state = np.zeros((n * m, n * m))
    for i in range(l):
        tensor = np.kron(np.array(qutip.rand_dm(n)), np.array(qutip.rand_dm(m)))
        separable_state = np.add(separable_state, prob[i] * tensor)
    return separable_state


def get_separable_states(num: int, n: int, m: int, e: bool = True) -> np.ndarray:
    separable_states = []
    for i in range(num):
        separable_states.append(embed(get_separable_state(n, m), e))
        print(i)
    return separable_states
    return np.array(separable_states)


def get_random_states(num: int, n: int, m: int, e: bool = True) -> np.ndarray:
    random_states = []
    for i in range(num):
        random_states.append(embed(np.array(qutip.rand_dm(n * m)), e))
        print(i)
    return random_states
    return np.array(random_states)

In [11]:
data = get_separable_states(40000, 2, 2, False)
random_data = get_random_states(30000, 2, 2, False)
cnt = 0
for matrix in random_data:
    if cnt == 10000:
        break
    if PPT(matrix, 2, 2) == 1:
        cnt += 1
        data.append(matrix)
percentage = 40000/(cnt + 40000)
np.save(f"{percentage}-sep-data-2x2", np.array(data))

## Classification Models

The following set of programs deal with models that we shall use to classify between separable and entangled states. The plan is to beat the state of the art claim of 97.5% accuracy in low dimensional cases.



*   0: SEPARABLE
*   1: ENTANGLED



In [16]:
from pyod.models.ocsvm import OCSVM
from pyod.models.so_gaal import SO_GAAL
from pyod.models.anogan import AnoGAN
from pyod.utils.example import visualize
import numpy as np


def accuracy(a, b):
    n = range(len(a))
    l = len(a)
    return (l - sum([a[i] != b[i] for i in n])) / l * 100


def load_data(n: int, m: int, file: str, e: bool = False):
    X_train = np.load(file)
    if bool:
        return embedding_matrices(X_train)
    return X_train


class Model:
    def __init__(self, n: int, m: int, file: str) -> None:
        self.X = load_data(n, m, file)

    def ocsvm(self, contamination):
        classifier = OCSVM(nu=0.9, gamma=0.5, kernel="rbf", verbose=1, contamination=contamination)
        classifier.fit(X=self.X)
        return classifier

    def gaal(self, contamination):
        classifier = SO_GAAL(contamination=contamination, stop_epochs=10)
        classifier.fit(X=self.X)
        return classifier

    def anogan(self, contamination):
        classifier = AnoGAN(contamination=contamination)
        classifier.fit(X=self.X)
        return classifier

## Classification of $2 \otimes 2$ states

Here, we try to classify $2 \otimes 2$ quantum states into separable and entangled classes. Remember, in this case we can use the PPT criterion as a necessary and sufficient condition. 

In [23]:
model = Model(2, 2, f"/content/0.8-sep-data-2x2.npy")
classifier = model.ocsvm(0.2)

[LibSVM]

### Prediction is done as follows.

In [69]:
TESTCASES = 1000

In [70]:
X_test1 = get_separable_states(TESTCASES, 2, 2, False)
X_test2 = get_random_states(TESTCASES, 2, 2, False)

In [71]:
y_test1 = check_PPT(X_test1, 2, 2)
y_test2 = check_PPT(X_test2, 2, 2)

In [72]:
print(y_test1)
print(y_test2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [73]:
pred_test1 = classifier.predict(embedding_matrices(X_test1))
pred_test2 = classifier.predict(embedding_matrices(X_test2))

In [74]:
print(pred_test1)
print(pred_test2)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

### Analysis is done as follows.

In [75]:
conf_test1 = classifier.predict_confidence(embedding_matrices(X_test1))
conf_test2 = classifier.predict_confidence(embedding_matrices(X_test2))

prob_test1 = classifier.predict_proba(embedding_matrices(X_test1))
prob_test2 = classifier.predict_proba(embedding_matrices(X_test2))

In [76]:
correct = 0
for i in range(len(y_test1)):
    if y_test1[i] == pred_test1[i]:
        correct += 1
    # else:
    #     print(conf_test1[i], prob_test1[i])
print(f"{correct / TESTCASES * 100}%")

95.6%


In [77]:
correct = 0
for i in range(len(y_test2)):
    if y_test2[i] == pred_test2[i]:
        correct += 1
    # else:
    #     print(conf_test2[i], prob_test2[i])
print(f"{correct / TESTCASES * 100}%")

80.5%
